In [2]:
#import required libraries
import requests
import json
import re
import time

In [3]:
#load your personal API key
with open('private_key/guardian_key.txt', 'r') as file:
    key = file.read().strip()
len(key)

36

In [4]:
 #build a search URL
base_url = 'https://content.guardianapis.com/'
search_string = "(ChatGPT OR AI) AND Risks AND Society"
#production_office = "aus"
from_date = "2023-12-01"
#to_date = "2023-12-31"
#full_url = base_url+f"search?q={search_string}&from-date={from_date}&to-date={to_date}&show-fields=body&api-key={key}"
full_url = base_url+f"search?q={search_string}&from-date={from_date}&show-fields=body&api-key={key}"
#url = baseUrl+'"'+searchString+'"'+'&production-office='+production_office+'&from-date='+fromDate+'&api-key='+key
print(full_url[:120])

https://content.guardianapis.com/search?q=(ChatGPT OR AI) AND Risks AND Society&from-date=2023-12-01&show-fields=body&ap


In [5]:
# get data from server
server_response = requests.get(full_url)
server_data = server_response.json()
resp_data = server_data.get('response','')
if resp_data == '':
    print("ERROR obtaining results:",server_data)
else:
    print("SUCCESS!")
    print(f"{resp_data['total']} results found available in {resp_data['pages']} pages")
    print(f"{resp_data['pageSize']} results per page")
    results = resp_data.get('results',[])

SUCCESS!
49 results found available in 5 pages
10 results per page


In [6]:
results[0]

{'id': 'technology/2024/apr/11/uk-has-real-concerns-about-ai-risks-says-competition-regulator',
 'type': 'article',
 'sectionId': 'technology',
 'sectionName': 'Technology',
 'webPublicationDate': '2024-04-11T14:57:28Z',
 'webTitle': 'UK has real concerns about AI risks, says competition regulator',
 'webUrl': 'https://www.theguardian.com/technology/2024/apr/11/uk-has-real-concerns-about-ai-risks-says-competition-regulator',
 'apiUrl': 'https://content.guardianapis.com/technology/2024/apr/11/uk-has-real-concerns-about-ai-risks-says-competition-regulator',
 'fields': {'body': '<p>Just six major technology companies are at the heart of the AI sector through an “interconnected web” of more than 90 investments and partnerships links, the UK’s competition regulator has warned, sparking increased concern about the anti-competitive nature of the technology.</p> <p>Sarah Cardell, the chief executive of the Competition and Markets Authority, said AI foundation models – general-purpose AI system

In [7]:
num_pages = resp_data['pages']
num_pages

5

In [8]:
def articles_from_page_results(page_results):
    articles = {}
    for result in page_results:
        article_date = result['webPublicationDate']
        article_title = result['webTitle']+f" [{article_date}]"
        article_html = result['fields']['body']
        article_text = re.sub(r'<.*?>','',article_html)
        articles[article_title] = article_text
    return articles

In [9]:
def get_all_articles_for_response(response_json,full_url):
    total_pages = response_json['pages']
    total_articles = response_json['total']
    print(f"Fetching {total_articles} articles from {total_pages} pages...")
    all_articles = {}
    page1_articles = articles_from_page_results(response_json['results'])
    all_articles.update(page1_articles)
    print("Added articles for page: 1")
    
    for page in range(2,total_pages+1):
        print("Getting articles from API for page:",page)
        page_response = requests.get(full_url+f"&page={page}")
        page_data = page_response.json()['response']
        print("Processing results for page:",page_data['currentPage'])
        page_articles = articles_from_page_results(page_data['results'])
        print(f"Fetched {len(page_articles)} articles.")
        all_articles.update(page_articles)
        print("Added articles for page:",page)
        print(f"Status: {len(all_articles)} articles.")
        time.sleep(1) # make sure we're not hitting the API to hard
    
    print(f"FINISHED: Fetched {len(all_articles)} articles.")
    return all_articles

In [10]:
my_articles = get_all_articles_for_response(resp_data,full_url)

Fetching 49 articles from 5 pages...
Added articles for page: 1
Getting articles from API for page: 2
Processing results for page: 2
Fetched 10 articles.
Added articles for page: 2
Status: 20 articles.
Getting articles from API for page: 3
Processing results for page: 3
Fetched 10 articles.
Added articles for page: 3
Status: 30 articles.
Getting articles from API for page: 4
Processing results for page: 4
Fetched 10 articles.
Added articles for page: 4
Status: 40 articles.
Getting articles from API for page: 5
Processing results for page: 5
Fetched 9 articles.
Added articles for page: 5
Status: 49 articles.
FINISHED: Fetched 49 articles.


In [11]:
print("Total Articles:",len(my_articles))
for title,text in my_articles.items():
    print(title)

Total Articles: 49
UK has real concerns about AI risks, says competition regulator [2024-04-11T14:57:28Z]
Seoul summit showcases UK’s progress on trying to make advanced AI safe  [2024-05-22T22:49:10Z]
Canada lawyer under fire for submitting fake cases created by AI chatbot [2024-02-29T20:10:03Z]
Don’t wait for Post Office-style scandal before regulating AI, ministers told [2024-02-06T00:01:46Z]
TechScape: Why is the UK so slow to regulate AI? [2024-02-06T11:53:22Z]
Is AI lying to me? Scientists warn of growing capacity for deception [2024-05-10T15:00:21Z]
AI-driven misinformation ‘biggest short-term threat to global economy’ [2024-01-10T09:52:18Z]
AI could be an extraordinary force for good. So why do our politicians still not have a plan? | Gaby Hinsliff [2024-03-08T06:00:44Z]
The Guardian view on the AI conundrum: what it means to be human is elusive | Editorial [2023-12-29T18:00:10Z]
AI doctors and chatbot nurses? Labour must show the future of the NHS isn’t so dystopian | Jeni Ten

In [12]:
file_path = "data/"
file_name = "AI risks to society.json"

with open(f"{file_path}{file_name}",'w', encoding='utf-8') as fp:
    fp.write(json.dumps(my_articles))